In [6]:
import pymongo

# Conexión al servidor MongoDB (reemplaza con tus datos)
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Nombre de la base de datos y colección
db_name = "Metadata"
collection_name = "Transcription"

# Obtener la base de datos y colección
db = client[db_name]
collection = db[collection_name]

# Realizar la consulta (obtiene todos los documentos)
cursor = collection.find()

data = []
# Iterar sobre los resultados e imprimirlos
for documento in cursor:
    data.append(documento)

len(data)


51

In [6]:
print(documento['Duration'])
for i in documento['Transcription']:
    print(i['Text'])

2748.48
 Perfecto.
 Pues muchas gracias por asistir a una nueva sesión de
 herramientas de visualización.
 Estamos con el primer gran bloque que tiene que ver con
 herramientas de visualización programáticas basadas en
 aplicaciones web.
 En este caso con HTML, JavaScript y D3,
 que es el framework que estamos utilizando y que,
 como hemos comentado en una sesión,
 nació y estaba en la red de la web.
 Y ahora estamos en la red de la web.
 Como hemos comentado en una sesión,
 nació y estaba totalmente orientado a la representación
 gráfica de contenido, ya sea texto,
 gráficos como veremos, incluye animaciones,
 incluye interacción.
 Y lo que vamos a ver hoy, la sesión se va a dividir en dos
 partes.
 Por un lado, la presentación de la actividad,
 que tiene que ver con la actividad grupal que se hace con
 D3.
 Y luego continuaremos con más ejercicios,
 con más elementos que hicimos, o sea,
 sobre el ejercicio que hicimos la semana pasada.
 Les voy a compartir mi pantalla.
 ¿Están viendo

In [3]:
print(documento['Duration'])
for i in documento['Transcription']:
    print(i['Text'])

2748.48
 PERTOR Pero tú puedearing ciertos
 co했던 hab
icating la
 que es el frenguo que estamos utilizando y que como hemos comentado
 de una sesión, nació y está totalmente orientado al representación gráfica
 de contenido, ya se atexo gráficos como veremos, incluyen invaciones,
 incluyen interactión. Y lo que vamos a ver hoy o la sesión se va a dividir
 en dos partes, por lo lado la presentación de interactividad,
 ¿vale?, que tiene que ver con la actividad global de que se ateconte
 de tres y luego continuaremos con más ejerterios con más elementos que hicimos
 o se sobre el ejerterio que hicimos la sesión a la base, ¿vale?,
 te voy a compartir mi pantalla,
 también no vi pantalla,
 perfecto, porque había varias preguntas o bastante preguntas y siempre he dudas
 alrededor de la actividad, ¿no?, como lo primero,
 la actividad simplemente había, creo que es importante explicarlo, ¿no?,
 el objetivo es o supuesto utilizar de tres, pero como siempre digo las herramientas
 de visuete en s

In [5]:
# Realizar la consulta (obtiene todos los documentos)
filter = {
    "FolderName":"Análisis e interpretación de datos",
    "FileName":"88730.mp4"
    }
cursor = collection.find(filter)

# Iterar sobre los resultados e imprimirlos
for documento in cursor:
    print(documento)

{'_id': ObjectId('666f674d9abf7c1994f8ddbb'), 'FileName': '88730.mp4', 'FolderName': 'Análisis e interpretación de datos', 'Duration': 600, 'SizeMb': 5.9326171875, 'Transcription': [{'Id': 0, 'StartTime': 0.0, 'EndTime': 9.8, 'Text': ' Muchísimas gracias. Me lié, me lié preguntando, me lié preguntando, eso las dudas de'}, {'Id': 1, 'StartTime': 9.8, 'EndTime': 13.56, 'Text': ' la anterior sesión y se volvió a menos, bueno, me lo que me preguntó.'}, {'Id': 2, 'StartTime': 13.56, 'EndTime': 21.8, 'Text': ' ¿Qué es el piefale? Vale, bueno, nos empezaron de nuevo, creo que vamos a explicar'}, {'Id': 3, 'StartTime': 21.8, 'EndTime': 26.2, 'Text': ' esto que es lo importante, vale. El primero, esta es esta en mar, vale, lo que'}, {'Id': 4, 'StartTime': 26.2, 'EndTime': 33.2, 'Text': ' acabó de hacer los cabeceros están más, ahí problemas con el audio.'}, {'Id': 5, 'StartTime': 33.2, 'EndTime': 38.2, 'Text': ' Si se escuchan de nuevo.'}, {'Id': 6, 'StartTime': 38.2, 'EndTime': 45.879999999999

In [7]:
result = collection.delete_many(filter)
print(result.deleted_count, "documentos eliminados")

0 documentos eliminados


In [7]:
# El TranscriptId que estás buscando
transcript_id = '4f9b6965-97d7-4c2a-90f3-015e098f4a53'

# Consulta para encontrar la transcripción específica
resultado = collection.find_one(
    {'Transcription.TranscriptId': transcript_id},
    {
        'Duration':1,
        'FileName':1,
        'FolderName':1,
        'Transcription.$': 1 
    }  # Proyección para devolver solo la transcripción específica
)

# Imprimir el resultado
print(resultado)

{'_id': ObjectId('666f9b50da1ca2495a147728'), 'FileName': '86970.mp4', 'FolderName': 'Análisis e interpretación de datos', 'Duration': 600, 'Transcription': [{'Id': 0, 'TranscriptId': '4f9b6965-97d7-4c2a-90f3-015e098f4a53', 'StartTime': 60.0, 'EndTime': 74.46000000000001, 'Text': ' Bueno, pues podemos empezar. Soy Jara Mesquita, voy a hacer gusto profesor en la'}]}


In [2]:
from functions.chroma_utils import *
collection_chroma = init_chroma('Transcription')
total_count = collection_chroma.count()
print(f"Total de elementos en la colección: {total_count}")
search_chroma(collection_chroma, ['sobre 7 hombres, lo las mujeres'], {'FileId': 'f6a40845-711e-4394-b19b-5cb68bb72017'}, 5)

Total de elementos en la colección: 26831
Total de elementos en la colección: 26831


{'ids': [[]],
 'distances': None,
 'metadatas': [[]],
 'embeddings': [[]],
 'documents': [[]],
 'uris': None,
 'data': None,
 'included': ['embeddings', 'metadatas', 'documents']}

In [5]:
from functions.chroma_utils import *  # Asegúrate de tener la función init_chroma() definida aquí


collection_chroma = init_chroma('Transcription')
total_count = collection_chroma.count()
print(f"Total de elementos en la colección: {total_count}")

# Realizar la búsqueda y almacenar los resultados
results = search_chroma(collection_chroma, ['en que clase se habla de mongo?'], {}, 5)

# Imprimir los resultados
if results['ids']:  # Verificar si hay resultados
    for i, document in enumerate(results['documents']):
        print(f"Resultado {i+1}: {document}")

        # Imprimir metadatos si es necesario (ajusta las claves según tus metadatos)
        print("Metadatos:", results['metadatas'][i])
else:
    print("No se encontraron resultados.")


Total de elementos en la colección: 26831
Total de elementos en la colección: 26831
Resultado 1: [' Yo no tengo que entrar en la mongo pedido,', ' en maramo, dice el mongo, que voy a usar el', ' pasadas en mongo, te diría que pasaba por grajar,', ' hablando del hackio de monco, en el empecéntaje un poco nervioso con ese tema, así que espero', ' Mongo que esta institución, este ejecutable,']
Metadatos: [{'EndTime': 194.46, 'FileId': 'cd1c7f25-6df4-46ad-ba25-ca4f589976b8', 'FileName': '116541.mp4', 'FolderName': 'Bases de Datos para el Big Data', 'Id': 43, 'StartTime': 191.46}, {'EndTime': 238.36, 'FileId': '9ac2ca21-5378-4da1-b3fb-cf962f8d3574', 'FileName': '91686.mp4', 'FolderName': 'Bases de Datos para el Big Data', 'Id': 58, 'StartTime': 235.36}, {'EndTime': 235.36, 'FileId': '9ac2ca21-5378-4da1-b3fb-cf962f8d3574', 'FileName': '91686.mp4', 'FolderName': 'Bases de Datos para el Big Data', 'Id': 57, 'StartTime': 233.16}, {'EndTime': 34.839999999999996, 'FileId': '44fc89b0-a27f-42ae-80f

In [1]:
from functions.mongo_utils import *
collection_mongo = init_mongodb("Metadata", "Transcription")
filter = {
                "FolderName": 'Análisis e Interpretación de Datos',
                "FileName": '88730.mp4'
            }
result = query_mongodb(collection_mongo, filter)
len(result)

1